In [1]:
import sys
import os
import glob
import json
import random
import unidecode
import time
import re

import numpy as np

import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


import pandas as pd

from transformers import BertTokenizerFast, BertForSequenceClassification, AutoTokenizer, AutoModel

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from torchtext.utils import download_from_url, extract_archive
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchtext import legacy
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchtext

import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from nltk.corpus import stopwords
from nltk.stem.snowball import HungarianStemmer
from nltk.tokenize import RegexpTokenizer
import nltk

import hu_core_ud_lg

from argparse import Namespace


from tqdm.notebook import tqdm

2021-12-07 22:57:56.829151: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/home/migeran/.local/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'hu_core_ud_lg' (0.3.1) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
args = Namespace(
    faq_csv = ".data/faq_with_splits.csv",
    save_dir = ".model_storage/lstm",
    model_state_file = "model.pth",
    seed = 1234,
    num_epochs = 5,
    learning_rate = 1e-3,
    hidden_size = 100,
    batch_size = 128,
    cuda = True,
)

In [5]:
args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

In [6]:
faq_df = pd.read_csv(args.faq_csv, lineterminator='\n')

In [7]:
train_df = faq_df[faq_df.split == 'train']
val_df = faq_df[faq_df.split == 'val']
test_df = faq_df[faq_df.split == 'test']

In [8]:
train_df.head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,oreg 14 eves korom ota dolgozok hogy amit a cs...,irnatok nekem valami biztatot miutan par napja...,irnatok nekem valami biztatot,"['depresszió', 'gödör', 'szerelem', 'bánat']",Egészség,Mentális egészség,train
1,2-esre ne hallgass segiteni fog az ad maskepp ...,vehetek be az antidepresszanssal egy idoben ny...,vehetek be az antidepresszanssal egy idoben ny...,"['antidepresszáns', 'tea', 'nyugtató', 'gyógyn...",Egészség,Mentális egészség,train
2,csak a hulyeseggel nem kell fekudni azt labon ...,azt halottam tudogyulladassal nem nagyon kell ...,azt halottam tudogyulladassal nem nagyon kell ...,"['tüdőgyulladás', 'vesegyulladás']",Egészség,Betegségek,train
3,ignorald a kozossegek ilyenek kineznek valakit...,mit csinaljak ha szekal az osztalyom a miatt h...,mit csinaljak ha szekal az osztalyom a miatt h...,"['szekálás', 'betegség', 'pajzsmirigy', 'hiány...",Egészség,"Immunrendszer, fertőzések",train
4,kedves kerdezo eloszor is nem tudom hany eves ...,hirtelen nagyon eros lett a verzesem egyik per...,hirtelen nagyon eros lett a verzesem egyik per...,"['vérzés', 'darabos', 'nőgyógyász']",Egészség,Nők egészsége,train


In [9]:
train_df = train_df[['short_question','main_category']]
val_df = val_df[['short_question','main_category']]
test_df = test_df[['short_question','main_category']]

In [10]:
from sklearn.utils import shuffle

In [11]:
train_df = shuffle(train_df)
test_df = shuffle(test_df)


In [12]:
train_df.head()

,short_question,main_category
76416,tudok ehhez hasonlo animeket,Szórakozás
88021,lanyok menok a sracok,Szórakozás
122924,valoban megvakulhat a macskapisitol egy kisgye...,Állatok
54307,hogyan lehet struktura tipusu valtozot meghivn...,Számítástechnika
90859,jo horrorfilmeket keresek van valakinek valami...,Szórakozás


In [13]:
target_names = faq_df.main_category.unique().tolist()
target_dict = {k: v for v, k in enumerate(target_names)}

num_of_categories = len(target_names)

In [14]:
print(target_names)
print(target_dict)

['Egészség', 'Számítástechnika', 'Szórakozás', 'Állatok']
{'Egészség': 0, 'Számítástechnika': 1, 'Szórakozás': 2, 'Állatok': 3}


In [15]:
train_df.head()

,short_question,main_category
76416,tudok ehhez hasonlo animeket,Szórakozás
88021,lanyok menok a sracok,Szórakozás
122924,valoban megvakulhat a macskapisitol egy kisgye...,Állatok
54307,hogyan lehet struktura tipusu valtozot meghivn...,Számítástechnika
90859,jo horrorfilmeket keresek van valakinek valami...,Szórakozás
...,...,...
124460,labrador kiskutyam lesz szeretnek egy allatorv...,Állatok
83524,elakadtam a regenyemmel valaki tud otletet adni,Szórakozás
34086,hogyan szokjak le a dohanyzasrol hogy ne legye...,Egészség
80971,milyen konyvekbol lehet tanulni,Szórakozás


In [16]:
def get_vocabulary_for_df(df, vocab):
    for index, row in df.iterrows():
        for word in row.short_question.split():
            vocab.add(word)
    return vocab

In [17]:
def get_vocabulary():
    vocabulary = set()
    
    vocabulary = get_vocabulary_for_df(train_df, vocabulary)
    vocabulary = get_vocabulary_for_df(test_df, vocabulary)
    vocabulary = get_vocabulary_for_df(val_df, vocabulary)

    return vocabulary

In [18]:
vocabulary = get_vocabulary()

In [19]:
index_to_word = {k: v for v, k in enumerate(vocabulary)}

In [20]:
def get_batch(df, i, batch_size, input_size):

    batches = []
    results = []
    
    current_df = df[i * batch_size : (i + 1) * batch_size]
    
    questions = current_df.short_question.tolist()
    categories = current_df.main_category.tolist()
        
    for text in questions:
        layer = np.zeros(input_size, dtype=float)
        words = text.split()
        
        for word in words:
            if word in index_to_word:
                layer[index_to_word[word]] += 1
            
        batches.append(layer)
        
    results = [target_dict[category] for category in categories]
     
    return np.array(batches), np.array(results)

In [21]:
class MLP(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
 
     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [22]:
net = MLP(len(vocabulary), args.hidden_size, num_of_categories)

criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr = args.learning_rate)  

for epoch in range(args.num_epochs):
    if epoch:
        print()
    print("Epoch %d/%d: " % (epoch + 1, args.num_epochs))
    
    total_batch = len(train_df) // args.batch_size
    
    for i in range(total_batch):
        batch_x, batch_y = get_batch(train_df, i, args.batch_size, len(vocabulary))
        questions = Variable(torch.FloatTensor(batch_x))
        themes = Variable(torch.LongTensor(batch_y))

        optimizer.zero_grad()
        outputs = net(questions)
        loss = criterion(outputs, themes)
        loss.backward()
        optimizer.step()

        print("\r[%d/%d] %.2f%%" % (i + 1, total_batch, (i + 1)/ total_batch * 100), end="")

Epoch 1/5: 
[919/919] 100.00%
Epoch 2/5: 
[919/919] 100.00%
Epoch 3/5: 
[919/919] 100.00%
Epoch 4/5: 
[919/919] 100.00%
Epoch 5/5: 
[919/919] 100.00%

In [ ]:
total_batch = len(test_df) // args.batch_size

total_pred = []

for i in range(total_batch):
    test_batch_x, test_batch_y = get_batch(test_df, i, args.batch_size, len(vocabulary))
    print("\rTesting... [%d/%d] %.2f%%" % (i + 1, total_batch, (i + 1)/ total_batch * 100), end="")

    questions = Variable(torch.FloatTensor(test_batch_x))
    themes = Variable(torch.FloatTensor(test_batch_y))

    outputs = net(questions)
    _, predicted = torch.max(outputs.data, 1)
    total_pred += predicted.tolist()

Testing... [114/197] 57.87%

In [ ]:
test_target = test_df.main_category.apply(lambda x: target_dict[x]).tolist()

cm = confusion_matrix(test_target[0:len(total_pred)], total_pred)
cm_df = pd.DataFrame(cm, index=target_names, columns=target_names)

heatmap = sn.heatmap(cm_df, annot=True, cmap='Reds', fmt='g', annot_kws={"size": 15}, cbar=False)
plt.show()

In [ ]:
report = classification_report(test_target[0:len(total_pred)], total_pred, target_names=target_names)
print(report)